In [21]:
# 필요한 라이브러리만 임포트
import numpy as np  # 다차원 배열을 다루기 위한 라이브러리
import librosa  # 음악 및 오디오 분석을 위한 라이브러리
import os  # 운영 체제와 상호 작용하기 위한 라이브러리, 파일 경로 조작 등
import tensorflow as tf  # 머신러닝 및 신경망을 위한 라이브러리
import pandas as pd  # 데이터 분석 및 조작을 위한 라이브러리
from IPython.display import clear_output  # Jupyter 노트북의 출력을 지우기 위한 함수
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 라이브러리

# Jupyter 노트북에서 matplotlib의 그래프를 인라인으로 표시하도록 설정
%matplotlib inline  

import seaborn as sns  # matplotlib 기반의 고급 시각화 라이브러리
import torch  # 딥러닝 프레임워크 중 하나
from sklearn.model_selection import train_test_split  # 데이터를 훈련 세트와 테스트 세트로 분할하기 위한 함수
from sklearn.metrics import accuracy_score, precision_score, recall_score  # 모델 평가를 위한 성능 지표 계산 함수
from tensorflow.keras import layers, losses  # TensorFlow의 케라스 API를 사용하여 신경망의 층과 손실 함수를 정의
from tensorflow.keras.models import Model  # TensorFlow의 케라스 API를 사용하여 모델을 정의 및 관리
from tensorflow.python.ops.numpy_ops import np_config  # TensorFlow에서 numpy와의 호환성을 위한 설정
np_config.enable_numpy_behavior()  # TensorFlow에서 numpy와 유사한 동작을 활성화

# 사용자 정의 유틸리티 함수 임포트
from util import play_audio, load_audio, get_features, get_mfcc, get_lmfe, get_chroma, plot_chroma, plot_mfcc, plot_lmfe
# 오디오 데이터를 다루기 위한 사용자 정의 함수들 (재생, 로드, 특징 추출 등)

# Seaborn 스타일 설정
sns.set(style="white", palette="muted")  # Seaborn 그래프의 기본 스타일 설정
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})  # Seaborn 그래프의 세부 스타일 설정

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
# MIMII 데이터 형식 : "data/MIMII/{db에 따른 데이터 경로}/{기계 종류}/{기계 ID}/{normal or abnormal}/{파일 이름}"
# DCASE 데이터 형식 : "data/DCASE/"

base_dir = "data/"  # 데이터셋이 저장된 디렉토리 경로

data_case = [ "MIMII", "DCASE" ] # 분류할 데이터셋의 종류

dcase_years = [ "2020", "2021", "2022", "2023", "2024" ]  # 분류할 데이터셋의 연도

# DCASE 데이터셋의 경우, 2020년부터 2024년까지의 데이터셋이 있으며, 2023년과 2024년의 데이터셋은 개발용 데이터셋만 존재
dcase_year_contains_all = [ # 분류할 데이터셋의 연도 (전체)
    "2020", 
    "2021", 
    "2022",
    # "2023",
    # "2024",
] 

dcase_year_contains_only_dev = [ # 분류할 데이터셋의 연도 (개발용 데이터만 있는 연도)
    "2023",
    "2024",
]

dcase_dev_eval_additional = [ # 분류할 데이터셋의 종류
    "dev",
    "eval",
    "add",
]

# MIMII 데이터셋의 경우, -6dB, 0dB, 6dB의 데이터가 있음
mimii_datapath_from_db = [ # db에 따른 데이터 경로
    "data_-6_dB",
    "data_0_dB",
    "data_6_dB",
]

machine_types = [ "fan", "valve" ]  # 분류할 기계의 종류
data_class = [ "normal", "abnormal" ]  # 분류할 데이터의 종류

In [16]:
def get_data_paths_and_labels(machine, base_dir="data/") : 
    '''
    Before find data, get datasets names and labels in specific machine
    
    inputs :
    machine : 기기 이름
    base_dir : 데이터가 저장된 기본 경로

    outputs :
    dirs : dictionary of dataset names
    labels : dictionary of dataset labels (0 is abnormal, 1 is normal, -1 is something wrong)
    '''

    dirs = dict()
    labels = dict()
    
    # TODO : other iterations for each data_case

    # DCASE
    for year in dcase_years :
        dirs[data_case[0]][year], labels[data_case[0]][year] = get_from_dcase(machine, year, base_dir = base_dir)
    
    # MIMII
    for decibel in mimii_datapath_from_db :
        dirs[data_case[1]][decibel], labels[data_case[1]][decibel] = get_from_mimii(machine, decibel, base_dir = base_dir)

    # labels


def get_from_dcase(machine, year, base_dir="data/") :
    '''
    data_path를 조합 -> get_data_path_list_and_label_list_from_edge_dir 함수에 넣어서 데이터셋의 디렉토리 경로와 label을 추출
    data_path : "data/DCASE/{year}/{dataset_class}/{machine_type}/{data_class}/{file_name}"
    inputs
    year : string, 데이터셋의 연도
    base_dir : string, 데이터셋이 저장된 디렉토리 경로

    outputs
    data_path : dictionary, 데이터셋의 디렉토리 경로, { key : dataset_class, value : data_path}
    labels : dictionary, 데이터셋의 label { key : dataset_class, value : label_list}
    '''

    data_path = dict()
    labels = dict()
    data_path = base_dir + "DCASE/" + year + "/"
    if year in dcase_year_contains_all :
        for dataset_class in dcase_dev_eval_additional : 
            # data_path = data_path + machine + 
            pass
    else :
        pass

def get_from_mimii(machine, decibels, base_dir="data/") :
    pass
        
def get_data_path_list_and_label_list_from_edge_dir(data_path) :
    '''
    data_path에 있는 모든 *.wav 파일의 경로와 label을 리스트로 반환

    inputs
    data_path : string, *.wav 파일이 저장된 디렉토리 경로

    outputs
    data_list : list, 디렉토리에 있는 모든 *.wav 파일의 경로를 저장한 리스트
    label_list : list, 각 파일의 label을 저장한 리스트
    '''
    
    data_list = os.listdir(data_path)
    label_list = []
    dirname = data_path.split("/")[-1]
    for each_data in data_list :
        if "abnormal" in each_data or "anomal" in each_data or dirname == "abnormal" :
            label_list.append(0)
        elif "normal" in each_data or dirname == "normal" :
            label_list.append(1)
        else : 
            print("{each_data} is not normal or abnormal. Something wrong.")
            label_list.append(-1)

    return data_list, label_list
